In [1]:
import numpy as np
import igl
import meshplot as mp
import random
import copy
import math

In [2]:
# import model + display it
v, f = igl.read_triangle_mesh('data/bone_rotate.obj')

  o bone


In [3]:
v -= v.min(axis=0)
v /= v.max()
igl.write_triangle_mesh("data/bone_unify.obj", v, f)

True

In [4]:
mp.plot(v, shading={"point_size": 0.03})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [5]:
# find the bounding box, draw the bounding box
m = np.min(v, axis=0)
ma = np.max(v, axis=0)

# Corners of the bounding box
v_box = np.array([[m[0], m[1], m[2]], [ma[0], m[1], m[2]], [ma[0], ma[1], m[2]], [m[0], ma[1], m[2]],
                  [m[0], m[1], ma[2]], [ma[0], m[1], ma[2]], [ma[0], ma[1], ma[2]], [m[0], ma[1], ma[2]]])

# Edges of the bounding box
f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6, 7], 
                  [7, 4], [0, 4], [1, 5], [2, 6], [7, 3]], dtype=np.int32)

p = mp.plot(v, shading={"point_size": 0.03})



p.add_edges(v_box, f_box, shading={"line_color": "green"});
p.add_points(v_box, shading={"point_size": 0.1, "point_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

2

In [6]:
# segments_recording = np.zeros(len(v), dtype=int)

# for i in range(len(v)):
#     item = v[i]
#     if item[1]< 0.5 :
#         segments_recording[i] = 1
        
# mp.plot(v, c = segments_recording, shading={"point_size": 0.03})

In [7]:
Face_0 = np.array([0,1,2,3])
Face_1 = np.array([4,5,6,7])
Face_2 = np.array([0,1,5,4])
Face_3 = np.array([1,2,5,6])
Face_4 = np.array([2,3,6,7])
Face_5 = np.array([0,3,7,4])

In [8]:
# a function, given face, v_box, return the selected center and radius


def find_circle(v_box, face):
    face_vs = v_box[face]
    print
    m_f = np.min(face_vs, axis=0)
    ma_f = np.max(face_vs, axis=0)


    center = np.zeros(3)
    radius = 1e+10
    direction = np.zeros(3)
    axis = 0
    for i in range(3):
        if m_f[i] == ma_f[i]:
            center[i] = (ma_f[i]+m_f[i])/2
            if center[i] > 0.5:
                direction[i] = - 1
            else:
                direction[i] = 1
            axis = i
        else:
            jiggle_1 = (random.random()-0.5)/10
            center[i] = (ma_f[i]+m_f[i])/2 + jiggle_1
            r1 = center[i] - m_f[i]
            r2 = ma_f[i] - center[i]
            jiggle_2 = random.random()/50
            radius = min(radius, min(r1, r2)) - jiggle_2 - 0.05

        
    return center, radius, direction, axis


def generate_circle_3d(center, radius, direction, num_points=100):
    direction = np.array(direction)
    direction = direction / np.linalg.norm(direction)  # normalize

    # Find two orthogonal vectors perpendicular to direction
    if np.allclose(direction, [0, 0, 1]):
        v1 = np.array([1, 0, 0])
    else:
        v1 = np.cross(direction, [0, 0, 1])
        v1 = v1 / np.linalg.norm(v1)
    
    v2 = np.cross(direction, v1)
    
    angles = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    circle_points = [
        center + radius * (np.cos(theta) * v1 + np.sin(theta) * v2)
        for theta in angles
    ]
    return np.array(circle_points)

center, radius, direction, axis = find_circle(v_box, Face_0)


p = mp.plot(v, shading={"point_size": 0.03})
p.add_edges(v_box, f_box, shading={"line_color": "green"});
p.add_points(v_box, shading={"point_size": 0.1, "point_color": "green"})
p.add_points(points=np.array([center]), shading={"point_size": 0.1, "point_color": "green"})

circle_points = generate_circle_3d(center, radius, direction, num_points=100)
p.add_points(circle_points, shading={"point_size": 0.1, "point_color": "green"})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

4

In [9]:
# a function, given center, radius, direction, return an array of points to occupy the circle

def circle_points(center, radius, direction, axis, resolution = 10):
    current_axis = 0
    circle_range = np.zeros((2, resolution))
    for i in range(3):
        if i != axis:
            circle_range[current_axis] = np.linspace(center[current_axis]-radius, center[current_axis]+radius, resolution)
        current_axis += 1

    result = []
    for cord_0 in circle_range[0]:
        for cord_1 in circle_range[1]:
            cord = np.array([cord_0, cord_1])
            cord = np.insert(cord, axis, 0)
            if np.linalg.norm(cord - center) < radius:
                result.append(cord)
    return np.array(result)
                

        
points = circle_points(center, radius, direction, axis, 100)
p = mp.plot(v, f,shading={"point_size": 0.03})
p.add_points(points, shading={"point_size": 0.01, "point_color": "green"})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

1

In [10]:
p = mp.plot(v, f)

points = circle_points(center, radius, direction, axis, 20)
p.add_points(points, shading={"point_size": 0.01, "point_color": "green"})


f_projected = set()
for point in points:
    point_cp = copy.deepcopy(point)
    result = igl.ray_mesh_intersect(point_cp, direction, v, f)
    if len(result) != 0:
        p.add_points(np.array([point_cp+direction*result[0][-1]]), shading={"point_size": 0.1, "point_color": "green"})
        new_f = result[0][0]
        if new_f not in f_projected:
            f_projected.add(new_f)

# print(f_projected)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [11]:
TT, TTi = igl.triangle_triangle_adjacency(f)

def expand_faces(TT, f_projected):
    new_f_projected = set()

    for face in f_projected:
        for adj_face in TT[face]:
            if adj_face not in new_f_projected:
                new_f_projected.add(adj_face)
    f_projected = f_projected.union(new_f_projected)
    return f_projected

for i in range(10):
    f_projected = expand_faces(TT, f_projected)
    print(len(f_projected))

380
620
814
981
1154
1321
1502
1706
1894
2093


In [12]:
def color_faces(v, f, f_set):
    colors = np.tile(np.array([0.7, 0.7, 0.7]), (f.shape[0], 1)).astype(np.float32)
    indices = np.fromiter(f_set, dtype=int)
    colors[indices] = np.array([0.0, 0.0, 1.0])
    print("Selected face indices:", colors)

    return mp.plot(v, f, c = colors)


color_faces(v, f, f_projected)

Selected face indices: [[0.7 0.7 0.7]
 [0.7 0.7 0.7]
 [0.7 0.7 0.7]
 ...
 [0.7 0.7 0.7]
 [0.7 0.7 0.7]
 [0.7 0.7 0.7]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [13]:
segments_recording = np.ones(len(v), dtype=int)

for i in f_projected:
    for j in f[i]:
        segments_recording[j] = 0

mp.plot(v, c = segments_recording, shading={"point_size": 0.03})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [14]:
# random select
# f_handle = random.sample(list(f_projected), 1)

# for i in f_handle:
#     segments_recording[f[i][0]] = 2


# mp.plot(v, c = segments_recording, shading={"point_size": 0.1})

In [15]:
points_constraint_circle = circle_points(center, radius*1.3, direction, axis, 5)

f_constraint_projected = set()
for point in points_constraint_circle:
    point_cp = copy.deepcopy(point)
    result = igl.ray_mesh_intersect(point_cp, direction, v, f)
    if len(result) != 0:
        p.add_points(np.array([point_cp+direction*result[0][-1]]), shading={"point_size": 0.1, "point_color": "green"})
        new_f = result[0][0]
        if new_f not in f_constraint_projected:
            f_constraint_projected.add(new_f)

index_recording = []
for i in f_constraint_projected:
    segments_recording[f[i][0]] = 2
    index_recording.append(f[i][0])

mp.plot(v, c = segments_recording, shading={"point_size": 0.1})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [16]:
np.save('data/bone.label.npy', segments_recording)

In [17]:
mp.plot(v, c = segments_recording, shading={"point_size": 0.1})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [18]:
# deform the class 2


def deform_constraint(v, index_recording, direction, axis, center):
    v_new = copy.deepcopy(v)

    for i in index_recording:
        cord = v_new[i]
        dif = cord - center
        dif[axis] = 0
        factor = np.linalg.norm(cord - center)/0.5

        
        v_new[i] = v_new[i] + math.sqrt((- factor**2 +1.2))*0.15 * direction
    return v_new

v_new = deform_constraint(v, index_recording, direction, axis, center)

mp.plot(v_new, c = segments_recording, shading={"point_size": 0.1})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

In [19]:
igl.write_triangle_mesh("data/bone_ditto.obj", v_new, f)

True

In [20]:
print(index_recording)

[2024, 2227, 12493, 2357, 12501, 7871, 12300, 7881, 7463]
